In [1]:
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome import options
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
import html5lib
import openpyxl
import datetime 
from datetime import date, timedelta
import pyodbc
import pymysql
import sqlalchemy as db
from sqlalchemy.orm import sessionmaker


In [2]:
usuario = 'guilherme.lobo'
password= 'Alfa2905'
options = Options()
#options.add_argument('--headless')
options.add_argument('True')
navegador = webdriver.Chrome(options=options)
navegador.implicitly_wait(10)
link='http://sifserver.gonsys.com:15001/SIF/authentication/login'
navegador.get(url=link)
navegador.implicitly_wait(5)
inputUsuario = navegador.find_element(by=By.ID,value='txtUser')
inputUsuario.send_keys(usuario)
inputSenha = navegador.find_element(by=By.ID,value='txtPassword')
inputSenha.send_keys(password)

navegador.implicitly_wait(5)
botao = navegador.find_element(by=By.NAME,value='sign-in')
botao.click()
navegador.implicitly_wait(5)

rps = navegador.find_element(by=By.LINK_TEXT,value= 'RPS')
rps.click()
navegador.implicitly_wait(10)

dropdow = navegador.find_element(By.ID,'ddlEmpresa')
dropdow = Select(dropdow)
dropdow.select_by_value('1')
navegador.implicitly_wait(5)

pesquisar = navegador.find_element(by=By.XPATH,value='/html/body/app-root/div[2]/app-rps/body/form/div[2]/button')
pesquisar.click()

# sem este sleep o resultado do get retornava vazio
sleep(4)

navegador.implicitly_wait(5)
resultado = navegador.find_element(by=By.XPATH,value='/html/body/app-root/div[2]/app-rps/body/div[2]/div/div/table')
navegador.implicitly_wait(10)
html = resultado.get_attribute("outerHTML")
navegador.implicitly_wait(6)
soup = BeautifulSoup(html,'html.parser')
relatorio = soup.find(name='table')
dataframe = pd.read_html(str(relatorio))[0]
#dataframe = dataframe
navegador.implicitly_wait(6)

prestacao = navegador.find_element(by=By.XPATH,value='//*[@id="navbarNavDropdown"]/ul/li[5]/a')
prestacao.click()
navegador.implicitly_wait(10)

DataAtual = date.today()
td = timedelta(-10)
entrada = DataAtual+td
entrada = entrada.strftime("%d/%m/%Y")

inputEntrada = navegador.find_element(by=By.ID,value='txtDtInicial')
inputEntrada.clear()
inputEntrada.send_keys(entrada)
navegador.implicitly_wait(10)

filtrar = navegador.find_element(by=By.XPATH,value='/html/body/app-root/div[2]/app-prestacao-conta-consultar/div[3]/div/form/fieldset/div/div[4]/button[1]')
filtrar.click()

navegador.implicitly_wait(5)
DataSif = navegador.find_element(by=By.XPATH,value='/html/body/app-root/div[2]/app-prestacao-conta-consultar/div[5]/div/table')
navegador.implicitly_wait(6)

PaginaSif = DataSif.get_attribute("outerHTML")
navegador.implicitly_wait(6)
Conversor = BeautifulSoup(PaginaSif,'html.parser')
ListaDatas = Conversor.find(name='table')
ListaConvert = pd.read_html(str(ListaDatas))[0]


data_atual = date.today()
data_atual = data_atual.strftime('%d/%m/%Y')

hora_atual = datetime.datetime.now()
hora_atual = hora_atual.strftime('%H:%M')

controle = dataframe.assign(Data = data_atual,Hora = hora_atual,Usuário= ['Guilherme'])

controle = controle.drop(columns=['Garagem','Empresa','Estado','Cidade'])


def acesso(query):
    server = '127.0.0.1,1434' 
    database = 'SIF_Prod' 
    username = 'sqlserver' 
    password = '@dm1nSIF'  
    cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
    cursor = cnxn.cursor()
    query = query
    dataframe_banco = pd.read_sql(query, cnxn)

    return dataframe_banco

dataframe_banco = acesso('select p.Descricao,max(DataEmissao) data from RPS r join Produto p on r.idproduto=p.IDProduto  group by r.IDProduto,p.Descricao')


rotativo = str(dataframe_banco.loc[0,'data'])
mens_pf = str(dataframe_banco.loc[1,'data'])
mens_pj = str(dataframe_banco.loc[2,'data'])
selos = str(dataframe_banco.loc[3,'data'])

controle = controle.assign(Rotativo=[rotativo],MensalistaPF=[mens_pf],MensalistaPJ=mens_pj,Selos=selos)
			
controle = controle.rename(columns={'Rotativo':'Ultima NFSe Rot'})
controle = controle.rename(columns={'MensalistaPF':'Ultima NFSe Men PF'})
controle = controle.rename(columns={'MensalistaPJ':'Ultima NFSe Men PJ'})
controle = controle.rename(columns={'Selos':'Ultima Nfse Selo'})
controle = controle[['Data','Hora','Usuário','Qtde RPS','Qtde Transmitido','Qtde Erro','Qtde Transmitir','Total Transmitido','Total com Erro','Total a Transmitir','Total Geral','Ultima NFSe Rot','Ultima NFSe Men PF','Ultima NFSe Men PJ','Ultima Nfse Selo']]


ColunaData=  ListaConvert.loc[:,'Data']
indice = len(ColunaData) - 1
ultimo = ColunaData[indice]

controle = controle

controle = controle.assign(ultima=ultimo)
			
controle = controle.rename(columns={'ultima':'Ultima Prestação'})

controle = controle[['Data','Hora','Usuário','Qtde RPS','Qtde Transmitido','Qtde Erro','Qtde Transmitir','Total Transmitido','Total com Erro','Total a Transmitir','Total Geral','Ultima NFSe Rot','Ultima NFSe Men PF','Ultima NFSe Men PJ','Ultima Nfse Selo','Ultima Prestação']]

controle.to_csv('relatorio.csv',index=False,encoding='latin-1',sep =';')
controle.to_excel('relatorio.xlsx',index=False,encoding='latin-1')

margem_prestacao = timedelta(1)
DataPrestacao = controle.loc[0,'Ultima Prestação']
CarenciaPrestacao = DataAtual+margem_prestacao
if DataPrestacao != CarenciaPrestacao :
    print('Prestação de Contas sem movimentação desde '+DataPrestacao+ ' Avisar Grupo Fiscal Indigo !!')
else:
    print('tudo ok')

qtde_erro = int(controle.loc[0,'Qtde Erro'])

#if qtde_erro > 0:
    #print('Reenviando Notas com erro!!')
#    acesso('''
#        UPDATE[dbo].[Rps]
#        SET [IDStatusTransmissao] = 1 
#        WHERE IDStatusTransmissao = 4
#        ''') 
#else:
#    print('Nunhum erro na fila de Rps\'s Tudo ok!!')

pd.read_excel('Acompanhamento SIF GRU.xlsx')

def leitor_xlsx(df):
    df_strd = pd.read_excel('Acompanhamento SIF GRU.xlsx')
    df= pd.concat([df_strd,df],ignore_index=True)
    writer = pd.ExcelWriter('Acompanhamento SIF GRU.xlsx',engine='xlsxwriter')
    df.to_excel(writer,sheet_name='Plan1',index=False)
    writer.close()

status_atual = pd.read_excel(r"relatorio.xlsx",sheet_name = 'Sheet1',)


leitor_xlsx(status_atual)

C:\Users\guilherme\AppData\Local\Temp\ipykernel_13100\3771034426.py:94: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataframe_banco = pd.read_sql(query, cnxn)
c:\Users\guilherme\OneDrive\Área de Trabalho\GUILHERME\PROJETOS PYTHON\Auto_Relatorio\.venv\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


Prestação de Contas sem movimentação desde Não há registros a serem exibidos. Avisar Grupo Fiscal Indigo !!


In [7]:
leitor_xlsx(status_atual)

In [8]:
from openpyxl import Workbook, load_workbook


In [ ]:
theFile = openpyxl.load_workbook('Acompanhamento SIF GRU.xlsx')
allSheetNames = theFile.sheetnames
print("All sheet names {} " .format(theFile.sheetnames))
def find_specific_cell():
    for row in range(1, currentSheet.max_row + 1):
        for column in "ABCDEFGHIJKL":  # Here you can add or reduce the columns
            cell_name = "{}{}".format(column, row)
            if currentSheet[cell_name].value == "Data":
                #print("{1} cell is located on {0}" .format(cell_name, currentSheet[cell_name].value))
                print("cell position {} has value {}".format(cell_name, currentSheet[cell_name].value))
                return cell_name
                
                

def get_column_letter(specificCellLetter):
    letter = specificCellLetter[0:-1]
    print(letter)
    return letter

def get_all_values_by_cell_letter(letter):
    for row in range(1, currentSheet.max_row + 1):
        for column in letter:
            cell_name = "{}{}".format(column, row)
            #print(cell_name)
            print("cell position {} has value {}".format(cell_name, currentSheet[cell_name].value))


for sheet in allSheetNames:
    print("Current sheet name is {}" .format(sheet))
    currentSheet = theFile[sheet]
    specificCellLetter = (find_specific_cell())
    letter = get_column_letter(specificCellLetter)

    get_all_values_by_cell_letter(letter)

    

In [44]:
cell_name

'F117'

In [3]:
import pyautogui as pag
import pyperclip

In [4]:

# Acesso automatizado dos relatório de selos do SIF
sleep(2)
pag.hotkey('Win','r')
sleep(2)
pyperclip.copy(r"C:\Users\guilherme\OneDrive\Área de Trabalho\GUILHERME\PROJETOS PYTHON\Auto_Relatorio\gerar_csv_selos.exe")
sleep(3)
pag.hotkey('Ctrl','v')
pag.press('Enter')
sleep(3)
DataAtual = date.today()
td = timedelta(-10)
Ent_Selos = DataAtual+td
Ent_Selos = Ent_Selos.strftime("%Y-%m-%d")
pyperclip.copy(Ent_Selos)
pag.hotkey('Ctrl','v')
pag.press('Enter')
sleep(10)
pag.press('Enter')

